In [16]:
from utils.envmap import generate_envmap
from torch import cat
from kornia.color import hsv_to_rgb

In [17]:
def normalize_image(light_latent, latent):
    # See: https://discuss.pytorch.org/t/current-torch-min-does-not-support-multiple-dimensions/55577/2
    if latent == 'scene-light':
        envmap_hs, envmap_v = light_latent.split([2, 512], dim=1)
        envmap_hsv = cat((envmap_hs.repeat_interleave(512, dim=1), envmap_v), dim=1).view(-1, 3, 16, 32)
        return 255. * hsv_to_rgb(envmap_hsv)
    elif latent == 'light':
        # TODO: it should not be normalized across channels
        envmap = light_latent.view(-1, 1536)
        envmap_min = envmap.min(dim=1)[0].unsqueeze(1).expand(-1, 1536)
        envmap_max = envmap.max(dim=1)[0].unsqueeze(1).expand(-1, 1536)
        return ((envmap - envmap_min) / (envmap_max - envmap_min)).view(-1, 3, 16, 32)

In [20]:
envmap_hsv = generate_envmap('N', 3500).unsqueeze(0)

In [22]:
normalize_image(envmap_hsv, 'scene-light').max()

tensor(241.3389)